# Capstone Project - The Battle of the City's 

### Applied Data Science Capstone by IBM/Coursera (WEEK 1)

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try assist a business woman (Restaurant Owner) of South African descend, who at her own capacity narrowed out the city of Toronto out of many that she would like to visit. Bearing in mind that Toronto is a big city and with only a limited time of stay, she only desires a specific destination that will satisfy her long time conquest of encorporating various cuisines to her african food chain of restaurants upon her return.
Our aim is to help her make an informed decision as to which geographical neighbourhood/s within Toronto will fulfil her busines goal, specifically. 

## Data <a name="data"></a>

Due to the nature of our problem at hand, we have outlined a couple of factors that we assumed will be crucial in our decision making:
* the total number of existing restaurants within the neighbourhoods in Toronto.
* Do any of these Neighbourhoods have any african restaurant and how many in total.
* the variety of restaurants would be esseentially crucial in our decision.

The following data sources will be needed to extract / generate the necessaray information:
* CSV file from wikipedia & (**from geopy.geocoders**) to convert an address into latitude and longitude values of Toronto neighbourhoods.
* in depth information of restuarants in every neighbourhood will be obtained using **FOURSQUARE API**

### City Option : Toronto

We will first look into or explore the city of Toronto using some web scrapping methods(wikipedia with csv files) to acquire area codes and cordinates using GEOPY GEOCODERS to give us specific adresses of neighbourhoods and subsequently restaurants of the city. 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis


import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


#### LOAD AND EXPLORE DATA

In [4]:
import requests

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

In [6]:
from bs4 import *

In [7]:
soup = BeautifulSoup(response.content, 'html5lib')
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en"><head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"badc2700-35a3-436a-b702-d512c0c42845","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1029579868,"wgRevisionId":1029579868,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","

#### Define Columns

In [8]:
table_contents=[]
table=soup.find('table')
for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
        
        
        
# print (table contents)

#### Load data into Data frame

In [9]:
df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 the Esplanade':'Downtown Toronto Stn A',
                                      'EEast TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                      'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                      'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# print (replace functions for getting Borough and Neighborhood information)

In [10]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [13]:
df.shape

(103, 3)

* **from the a data feedback we can confirm to our client the South African business woman that a total of 103 boroughs were listed from our web extraction using Canadas list of Postal Codes**

### We further explore and cluster neighbourhoods in Toronto specifically as admired by client (here we will do so using a CSV file)

In [11]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_075c3343398049d682782a459e61295b = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_075c3343398049d682782a459e61295b = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_075c3343398049d682782a459e61295b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HYe0QOJAk3fvqdpE90VRH3LHER16w1A5WgJN6D6-SOqu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_075c3343398049d682782a459e61295b)

body = client_075c3343398049d682782a459e61295b.get_object(Bucket='torontoneighbourhoods-donotdelete-pr-jojg4lyobwul7y',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### We then attach to data frame from csv file into our data frame

In [12]:
for data in df:
    
        
    
    df['Latitude'] = df_data_1['Latitude']
    df['Longitude'] = df_data_1['Longitude']
    
    
    
    
Toronto_hoods = df
Toronto_hoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476


In [16]:
Toronto_hoods.shape

(103, 5)

#### Upon acquiring cordinates of neighbourhoods in Toronto we can the visually give our client a glimpse of how the city looks like using our interesting data science tools and expertise.

In [13]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 8.3 MB/s  eta 0:00:01
  Created wheel for folium: file

In [14]:
import requests

#### With all our tools installed we can then go ahead and use geopy to further analyse 

In [15]:
address = 'Toronto, TOR'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7370584, -79.2442535.


#### And now to give our client a fun map of Toronto

In [16]:
# create map using latitude and longitude values
Toronto_hoods_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, neighborhood in zip(Toronto_hoods['Latitude'], Toronto_hoods['Longitude'], Toronto_hoods['Borough'], Toronto_hoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_hoods_map)  
    
Toronto_hoods_map

##### However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Downtown Toronto which we'l name DT_toronto. (This also came at discussions with client based on the map visuals above)

In [17]:
DT_toronto = Toronto_hoods[Toronto_hoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
4,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
5,M6G,Downtown Toronto,Christie,43.753259,-79.329656
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.695344,-79.318389
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.668999,-79.315572
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.689574,-79.383160


##### 17 neighbourhoods generated 

In [18]:
import requests

#### Lets get geographical coordinates of Downtown Toronto

In [19]:
adress = 'Down Toronto, TOR'

geolocator = Nominatim(user_agent= "tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.7370584, -79.2442535.


#### As we have down with the entire Toronto lets now only visualise Down Town Toronto as per client request

In [20]:
# create map of Downtown Toronto using latitude and longitude values
DT_toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DT_toronto['Latitude'], DT_toronto['Longitude'], DT_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(DT_toronto_map)  
    
DT_toronto_map

#### We then now define our FourSQUARE credentials and Version

In [21]:
CLIENT_ID = 'DSSTXARYPVOLXHCGDECIV2WREIH0IF5ZI3DVD4UMW4R24F5N' # your Foursquare ID
CLIENT_SECRET = 'TQW00CP2F2FM5Z3OYKP0KSEBKUOJRPLTUULJMLCLQXTBYCSK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


#### To provide our client with a subtle taste of what Down Town Toronto posses we further explored one of its Neighbourhoods in depth - **Regent Park, Harbourfront** 

In [22]:
DT_toronto.loc[0, 'Neighborhood'] # get neighborhood name

'Regent Park, Harbourfront'

In [23]:
# 1st lets get the coordinates
neighborhood_latitude = DT_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DT_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DT_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


#### We now create a GET request URL-name url!

In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=DSSTXARYPVOLXHCGDECIV2WREIH0IF5ZI3DVD4UMW4R24F5N&client_secret=TQW00CP2F2FM5Z3OYKP0KSEBKUOJRPLTUULJMLCLQXTBYCSK&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100'

#### Send GET request and also examine details

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d1b70e58238508cd53833a'},
 'response': {'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4beee041e24d20a1cd857314',
       'name': 'RBC Royal Bank',
       'location': {'address': '4374 KINGSTON RD',
        'crossStreet': 'Kingston & Lawrence',
        'lat': 43.76678992471017,
        'lng': -79.19115118872593,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.76678992471017,
          'lng': -79.1911511887

#### Run a function that extracts the category of the venue

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### for optimum resuslts let us clean the Json file and equip it into pd dataframe

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


#### As you can imagine our client is now eager to unveil more of what Toronto has to offer , with that being we will now explore all neighbourhoods in Downtown Toronto. So next we will create a function that to repeat the same process to all the neighbourhoods in Downtown Toronto. 

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### with above function we create a new dataframe called DT_venues

In [30]:
DT_venues = getNearbyVenues(names=DT_toronto['Neighborhood'],
                                  latitudes=DT_toronto['Latitude'],
                                  longitudes=DT_toronto['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


#### To provide our client with an idea of the vastness of our findings, we examine the size of our resulting dataframe

In [31]:
print(DT_venues.shape)
DT_venues.head()

(128, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


#### As you can imagine our client would essentially be more interested in how many venues were returned.

In [32]:
DT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,6,6,6,6,6,6
Christie,3,3,3,3,3,3
Church and Wellesley,8,8,8,8,8,8
"Commerce Court, Victoria Hotel",3,3,3,3,3,3
"First Canadian Place, Underground city",2,2,2,2,2,2
"Garden District, Ryerson",4,4,4,4,4,4
"Harbourfront East, Union Station, Toronto Islands",6,6,6,6,6,6
"Kensington Market, Chinatown, Grange Park",30,30,30,30,30,30


#### to provide her with a niche perspective of our results, we can then find out how many unique categories can be extracted from all returned venues

In [33]:
print('There are {} uniques categories.'.format(len(DT_venues['Venue Category'].unique())))

There are 71 uniques categories.


#### In total 71 unique venues were returned, with that being said we were certain our client would like us analyse each neighbourhood further(using the one hot coding method)

In [34]:
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

,Neighborhood,Airport,Athletics & Sports,Auto Workshop,Bank,Bar,Baseball Field,Beer Store,Breakfast Spot,Brewery,...,Skating Rink,South American Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Tennis Court,Trail,Vegetarian / Vegan Restaurant,Video Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# to check dataframe size 
DT_onehot.shape

(128, 72)

#### During our ongoing findings and constant communication with our client it was concluded that as much as our focal point was to discover the vasteness of the two city it would also be of great importance to also examine other surrounding and most frequent venues to assist our client further as to what other aspects attract people. (*To accomplish this we group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category.)*

In [36]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped

,Neighborhood,Airport,Athletics & Sports,Auto Workshop,Bank,Bar,Baseball Field,Beer Store,Breakfast Spot,Brewery,...,Skating Rink,South American Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Tennis Court,Trail,Vegetarian / Vegan Restaurant,Video Store,Yoga Studio
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,...,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.058824
1,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.0,0.333333,0.333333,0.000000,0.000000,0.000000
5,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.000000
8,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.033333,0.033333,0.0,0.000000,0.000000,0.000000,...,0.00,0.033333,0.000000,0.000000,0.1,0.000000,0.000000,0.033333,0.000000,0.033333
9,"Regent Park, Harbourfront",0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.111111,0.000000,...,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
# to confirm new size
DT_grouped.shape

(16, 72)

#### for further analysis and convienence we now print each neighbourhood along with 5 most common venues

In [38]:
num_top_venues = 5

for hood in DT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Garden Center  0.06
3                Park  0.06
4         Pizza Place  0.06


----Central Bay Street----
            venue  freq
0  Discount Store  0.17
1         Butcher  0.17
2     Pizza Place  0.17
3     Coffee Shop  0.17
4     Supermarket  0.17


----Christie----
                  venue  freq
0     Food & Drink Shop  0.33
1                  Park  0.33
2  Fast Food Restaurant  0.33
3     Health Food Store  0.00
4            Nail Salon  0.00


----Church and Wellesley----
                       venue  freq
0                Pizza Place  0.25
1             Discount Store  0.12
2  Middle Eastern Restaurant  0.12
3               Intersection  0.12
4                Coffee Shop  0.12


----Commerce Court, Victoria Hotel----
          venue  freq
0         Trail  0.33
1  Tennis Court  0.

#### Brilliant we now populate our findings into a pd dataframe but first we create a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### To give our client an intuitive analysis in a concise manner we then display the top 10 venues for each neighbourhood.

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Yoga Studio,Restaurant,Garden Center,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place
1,Central Bay Street,Coffee Shop,Pharmacy,Pizza Place,Supermarket,Discount Store,Butcher,Yoga Studio,Falafel Restaurant,Diner,Donut Shop
2,Christie,Fast Food Restaurant,Food & Drink Shop,Park,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Garden Center,Fish & Chips Shop,Gourmet Shop
3,Church and Wellesley,Pizza Place,Coffee Shop,Sandwich Place,Middle Eastern Restaurant,Chinese Restaurant,Discount Store,Intersection,Diner,Donut Shop,Electronics Store
4,"Commerce Court, Victoria Hotel",Trail,Tennis Court,Restaurant,Yoga Studio,Farmers Market,Curling Ice,Diner,Discount Store,Donut Shop,Electronics Store


In [42]:
neighborhoods_venues_sorted.shape

(16, 11)

#### For this project we chose the K-means CLUSTERING machine learning algorithm based on its accuracy and simplicity.